In [1]:
import numpy as np
import cv2
import torch
import torchvision
from PIL import Image
import os
import copy
import torch
import torchvision
import torch.nn as nn
import scipy
import torchvision.transforms as transforms
from torchvision import datasets as ds
from torch.utils.data import DataLoader,Dataset,Subset,ConcatDataset,random_split
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import tqdm
import celeba_dataset

In [2]:
print(os.getcwd())

/home/zhixin/Project/FL_CelebA_10_27


In [3]:
# 把数据缩放到（-1，1）
class Oneone(torch.nn.Module):
    def __init__(self, inplace=False):
        super().__init__()
        self.inplace = inplace

    def forward(self, tensor):
        return tensor*2.0-1.0
        # return F.normalize(tensor, self.mean, self.std, self.inplace)

# transform = transforms.Compose是把一系列图片操作组合起来，比如减去像素均值等。
# DataLoader读入的数据类型是PIL.Image
# 这里对图片不做任何处理，仅仅是把PIL.Image转换为torch.FloatTensor，从而可以被pytorch计算
transform_train = transforms.Compose([transforms.CenterCrop(224),
                                       transforms.Resize((128, 128)),
                                       #transforms.Grayscale(),
                                       #transforms.Lambda(lambda x: x/255.),
                                       transforms.ToTensor()])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    Oneone(),
])

In [4]:
learning_rate = 0.001
batch_size = 128

In [5]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}
intermediate_result = {}
net_name = "VGG16"

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512*4*4, 2)
        global intermediate_result

    def forward(self, x):
        seq = self.features
        out = x
        for i,layer in enumerate(seq):
            out = layer(out)

            if type(layer) == torch.nn.modules.conv.Conv2d:
                intermediate_result[str(i)] = out
#         out = self.features(x)
        out = out.view(out.size(0), -1)
        intermediate_result["linear"] = out
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]

                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)


In [6]:
net = VGG(net_name)
print(net)
# 定义损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
optimizer_2 = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)



# 如果有gpu就使用gpu，否则使用cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

#  加载CelebA数据集

In [7]:
#加载原始数据集
celebA_train_dataset=celeba_dataset.CelebA(root='../', split='train',target_type='attr', transform=transform_train, target_transform=None, download=True)
celebA_val_dataset=celeba_dataset.CelebA(root='../', split='test',target_type='attr',transform=transform_train, target_transform=None, download=True)

#取部分数据集
# celebA_train_dataset=random_split(celebA_train_dataset,
#                                   lengths=[int(len(celebA_train_dataset)*0.25),len(celebA_train_dataset)-int(len(celebA_train_dataset)*0.25)])[0]

celebA_train_loader=DataLoader(dataset = celebA_train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_loader=DataLoader(dataset = celebA_val_dataset,
                              batch_size=batch_size,
                              shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
print("标签参数如下")
for i,name in enumerate(celebA_val_dataset.attr_names):
    print(str(i)+" "+name)

print("训练集长度："+str(len(celebA_train_dataset)) )
print("验证集长度："+str(len(celebA_val_dataset)) )

标签参数如下
0 5_o_Clock_Shadow
1 Arched_Eyebrows
2 Attractive
3 Bags_Under_Eyes
4 Bald
5 Bangs
6 Big_Lips
7 Big_Nose
8 Black_Hair
9 Blond_Hair
10 Blurry
11 Brown_Hair
12 Bushy_Eyebrows
13 Chubby
14 Double_Chin
15 Eyeglasses
16 Goatee
17 Gray_Hair
18 Heavy_Makeup
19 High_Cheekbones
20 Male
21 Mouth_Slightly_Open
22 Mustache
23 Narrow_Eyes
24 No_Beard
25 Oval_Face
26 Pale_Skin
27 Pointy_Nose
28 Receding_Hairline
29 Rosy_Cheeks
30 Sideburns
31 Smiling
32 Straight_Hair
33 Wavy_Hair
34 Wearing_Earrings
35 Wearing_Hat
36 Wearing_Lipstick
37 Wearing_Necklace
38 Wearing_Necktie
39 Young
40 
训练集长度：162770
验证集长度：19962


In [9]:
#数据切分加工
def get_split_indices(dataset,target_index):
    '''
    按标签划分数据集，返回数据集的下标集合
    :param dataset:
    :param target_index: 按照这个下标的标签进行划分
    :return: 返回的list中，每个元素代表不同值的标签下标的集合
    '''
    group1=[]
    group2=[]
    for i,(data,target) in enumerate(dataset):
        if target[target_index]==0:
            group1.append(i)
        else:
            group2.append(i)
    return [group1,group2]
def shuffle_dataset(dataset,target_index):
    '''
    打乱数据集，把目标标签取反
    :param dataset:
    :param target_index: 标签下标
    :return:
    '''
    dataset=copy.deepcopy(dataset)
    for i,(data,target) in enumerate(dataset):
        dataset[i][1][target_index]=target[target_index]^1 #取反操作
    return dataset

In [10]:
#标签的下标
target_index=20
#身份标签的下标
group_index=20
#shuffle数据集的长度
shuffle_len=len(celebA_train_dataset)/10

In [18]:
indices=get_split_indices(celebA_train_dataset,target_index)
celebA_train_target1_dataset=Subset(celebA_train_dataset,indices[0])
celebA_train_target2_dataset=Subset(celebA_train_dataset,indices[1])
#构造一个标签平衡的训练集
length=15000
# celebA_train_dataset=ConcatDataset([random_split(celebA_train_target1_dataset,[length,len(celebA_train_target1_dataset)-length])[0],random_split(celebA_train_target2_dataset,[length,len(celebA_train_target2_dataset)-length])[0]])
# celebA_train_loader=DataLoader(dataset = celebA_train_dataset,
#                               batch_size=batch_size,
#                               shuffle=True)
print(len(celebA_train_target1_dataset))
print(len(celebA_train_target2_dataset))

KeyboardInterrupt: 

In [ ]:
#加载划分数据集
#男女验证集
indices=get_split_indices(celebA_val_dataset,group_index)
celebA_val_male1_dataset=Subset(celebA_val_dataset,indices[0])
celebA_val_male2_dataset=Subset(celebA_val_dataset,indices[1])
celebA_val_male1_loader=DataLoader(dataset = celebA_val_male1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_male2_loader=DataLoader(dataset = celebA_val_male2_dataset,
                              batch_size=batch_size,
                              shuffle=True)
#男女性训练集
indices=get_split_indices(celebA_train_dataset,group_index)
celebA_train_male1_dataset=Subset(celebA_train_dataset,indices[0])
celebA_train_male2_dataset=Subset(celebA_train_dataset,indices[1])

In [ ]:
#验证集
#性别1中biglips标签数据集
indices = get_split_indices(celebA_val_male1_dataset, target_index)
celebA_val_male1_biglips1_dataset = Subset(celebA_val_male1_dataset, indices[0])
celebA_val_male1_biglips2_dataset = Subset(celebA_val_male1_dataset, indices[1])
celebA_val_male1_biglips1_loader=DataLoader(dataset = celebA_val_male1_biglips1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_male1_biglips2_loader=DataLoader(dataset = celebA_val_male1_biglips2_dataset,
                              batch_size=batch_size,
                              shuffle=True)
#性别2中biglips标签数据集
indices = get_split_indices(celebA_val_male2_dataset, target_index)
celebA_val_male2_biglips1_dataset = Subset(celebA_val_male2_dataset, indices[0])
celebA_val_male2_biglips2_dataset = Subset(celebA_val_male2_dataset, indices[1])
celebA_val_male2_biglips1_loader=DataLoader(dataset = celebA_val_male2_biglips1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_male2_biglips2_loader=DataLoader(dataset = celebA_val_male2_biglips2_dataset,
                              batch_size=batch_size,
                              shuffle=True)
indices=get_split_indices(celebA_val_dataset,target_index)
celebA_val_biglips1_dataset=Subset(celebA_val_dataset,indices[0])
celebA_val_biglips2_dataset=Subset(celebA_val_dataset,indices[1])
celebA_val_biglips1_loader=DataLoader(dataset =celebA_val_biglips1_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_val_biglips2_loader=DataLoader(dataset =celebA_val_biglips2_dataset,
                              batch_size=batch_size,
                              shuffle=True)

In [ ]:
#训练集
#性别1中biglips标签数据集
indices=get_split_indices(celebA_train_male1_dataset,target_index)
celebA_train_male1_biglips1_dataset=Subset(celebA_train_male1_dataset,indices[0])
celebA_train_male1_biglips2_dataset=Subset(celebA_train_male1_dataset,indices[1])
#性别2中biglips标签数据集
indices=get_split_indices(celebA_train_male2_dataset,target_index)
celebA_train_male2_biglips1_dataset=Subset(celebA_train_male2_dataset,indices[0])
celebA_train_male2_biglips2_dataset=Subset(celebA_train_male2_dataset,indices[1])

In [34]:
#shuffle数据集，含50个男正例、反例和女性正例反例的数据集，并打乱biglips
# celebA_train_balance_dataset=ConcatDataset([Subset(celebA_train_male1_biglips1_dataset,range(100)),
#                                             Subset(celebA_train_male1_biglips2_dataset,range(100)),
#                                             Subset(celebA_train_male2_biglips1_dataset,range(100)),
#                                             Subset(celebA_train_male2_biglips2_dataset,range(100))] )
# celebA_train_balance_dataset=ConcatDataset([Subset(celebA_train_male1_dataset,range(100)),
#                                             Subset(celebA_train_male2_dataset,range(100))] )
# celebA_train_balance_dataset=ConcatDataset([random_split(celebA_train_male1_dataset,[100,len(celebA_train_male1_dataset)-100])[0],
#                                             random_split(celebA_train_male2_dataset,[100,len(celebA_train_male2_dataset)-100])[0]] )
celebA_train_balance_dataset=ConcatDataset([random_split(celebA_train_male1_biglips1_dataset,[int(shuffle_len/4),len(celebA_train_male1_biglips1_dataset)-int(shuffle_len/4)])[0],
                                            random_split(celebA_train_male1_biglips2_dataset,[int(shuffle_len/4),len(celebA_train_male1_biglips2_dataset)-int(shuffle_len/4)])[0],
                                            random_split(celebA_train_male2_biglips1_dataset,[int(shuffle_len/4),len(celebA_train_male2_biglips1_dataset)-int(shuffle_len/4)])[0],
                                            random_split(celebA_train_male2_biglips2_dataset,[int(shuffle_len/4),len(celebA_train_male2_biglips2_dataset)-int(shuffle_len/4)])[0]] )
celebA_train_shuffle_biglips_dataset=shuffle_dataset(celebA_train_balance_dataset,target_index=target_index)
celebA_train_balance_dataloader=DataLoader(dataset = celebA_train_balance_dataset,
                              batch_size=batch_size,
                              shuffle=True)
celebA_train_shuffle_biglips_dataloader=DataLoader(dataset = celebA_train_shuffle_biglips_dataset,
                              batch_size=batch_size,
                              shuffle=True)

In [12]:
# 训练模型的方法定义
print('training on: ', device)
def test(loader, net,target_index):
    net.eval()
    acc = 0.0
    sum = 0.0
    loss_sum = 0
    for batch, (data, target) in enumerate(loader):
        data, target = data.to(device), target[:,target_index].to(device)
        output = net(data)
        loss = criterion(output, target)
        loss_sum += loss.item()
        _, predicted = output.max(1)
        sum += target.size(0)
        acc += predicted.eq(target).sum().item()
        # acc += torch.sum(torch.argmax(output, dim=1) == target).item()
        # sum += len(target)
        # loss_sum += loss.item()
    print('test  acc: %.2f%%, loss: %.4f' % (100 * acc / sum, loss_sum / (batch + 1)))
    return 100 * acc / sum, loss_sum / (batch + 1)

def train(loader, model, target_index, training_type):
    '''
    :param loader:
    :param model:
    :param target_index: 标签下标
    :param training_type: 模型名称
    :return:
    '''
    model.train()
    acc = 0.0
    sum = 0.0
    loss_sum = 0

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
    for batch, (data, target) in tqdm.tqdm( enumerate(loader),desc="模型训练中：", total=len(loader)):
        data, target = data.to(device), target[:,target_index].type(torch.LongTensor).to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        _, predicted = output.max(1)
        sum += target.size(0)
        acc += predicted.eq(target).sum().item()

    print('train acc: %.2f%%, loss: %.4f' % (100 * acc / sum, loss_sum / (batch + 1)))
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "../models/" + str(training_type) + "_checkpoint.pth")

def load_model(model_path):
    #加载模型
    net = VGG('VGG16').to(device)
    checkpoint = torch.load(model_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    return net
def CelebA_test(model,target_index):
    print("全部测试集：")
    acc,loss=test(celebA_val_loader,model,target_index=target_index)
    # print("性别1测试集：")
    # test(celebA_val_male1_loader,model,target_index=target_index)
    # print("性别2测试集：")
    # test(celebA_val_male2_loader,model,target_index=target_index)
    # print("标签1测试集：")
    # test(celebA_val_biglips1_loader,model,target_index=target_index)
    # print("标签2测试集：")
    # test(celebA_val_biglips2_loader,model,target_index=target_index)
    return acc
def load_new_model():
    net = VGG(net_name)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    net= nn.DataParallel(net)
    return net

training on:  cuda


In [54]:
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    net= nn.DataParallel(net)

Using 2 GPUs


In [56]:
%%time
#原始训练
for epoch in range(200):
        print('epoch: %d' % epoch)
        train(celebA_train_loader,net,target_index=30,training_type="VGG16_origin_gender")
        acc=CelebA_test(net,target_index=30)
        if acc>90:
            print("epoch:"+str(epoch))
            break

epoch: 0


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1272/1272 [06:25<00:00,  3.30it/s]


train acc: 94.32%, loss: 0.2193
全部测试集：
test  acc: 95.31%, loss: 0.1729
epoch:0
CPU times: user 2h 14min 55s, sys: 5min 26s, total: 2h 20min 22s
Wall time: 7min 1s


In [16]:
# net=load_new_model()
# for epoch in range(200):
#         print('epoch: %d' % epoch)
#         train(celebA_train_loader,net,target_index=31,training_type="VGG16_origin_smiling")
#         acc=CelebA_test(net,target_index=31)
#         if acc>90:
#             print("epoch:"+str(epoch))
#             break
# net=load_new_model()
# for epoch in range(50):
#         print('epoch: %d' % epoch)
#         train(celebA_train_loader,net,target_index=6,training_type="VGG16_origin_biglips")
#         acc=CelebA_test(net,target_index=6)
#         if acc>90:
#             print("epoch:"+str(epoch))
#             break

# net=load_new_model()
# for epoch in range(25):
#         print('epoch: %d' % epoch)
#         train(celebA_train_loader,net,target_index=7,training_type="VGG16_origin_bignose")
#         acc=CelebA_test(net,target_index=7)
#         if acc>90:
#             print("epoch:"+str(epoch))
#             break
net=load_new_model()
for epoch in range(10):
        print('epoch: %d' % epoch)
        train(celebA_train_loader,net,target_index=33,training_type="VGG16_origin_wavyhair")
        acc=CelebA_test(net,target_index=33)
        if acc>90:
            print("epoch:"+str(epoch))
            break

epoch: 0


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1272/1272 [06:49<00:00,  3.11it/s]


train acc: 68.52%, loss: 0.6441
全部测试集：
test  acc: 63.68%, loss: 0.6398
epoch: 1


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1272/1272 [06:48<00:00,  3.12it/s]


train acc: 71.74%, loss: 0.5881
全部测试集：
test  acc: 70.04%, loss: 0.5826
epoch: 2


...

train acc: 74.39%, loss: 0.5243
全部测试集：
test  acc: 70.96%, loss: 0.5640
epoch: 8


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1272/1272 [06:48<00:00,  3.12it/s]


train acc: 74.60%, loss: 0.5163
全部测试集：
test  acc: 71.21%, loss: 0.5654
epoch: 9


模型训练中：: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1272/1272 [06:48<00:00,  3.11it/s]


train acc: 74.94%, loss: 0.5106
全部测试集：
test  acc: 72.10%, loss: 0.5440


In [ ]:
net=load_model('../models/VGG16(224*224)_origin_smiling_checkpoint.pth')

In [55]:
CelebA_test(net,target_index=target_index)

全部测试集：
test  acc: 71.73%, loss: 0.7059
性别1测试集：
test  acc: 68.92%, loss: 0.7069
性别2测试集：
test  acc: 75.67%, loss: 0.6967
标签1测试集：
test  acc: 84.75%, loss: 0.6167
标签2测试集：
test  acc: 59.11%, loss: 0.7674


In [56]:
#混淆训练,celebA数据集target为biglips
for epoch in range(2):
        print('epoch: %d' % epoch)
        train(celebA_train_shuffle_biglips_dataloader,net,target_index=target_index,training_type="VGG16(224*224)_shuffle_smiling")

epoch: 0


模型训练中：: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [01:27<00:00,  2.91it/s]


train acc: 64.74%, loss: 0.6459
epoch: 1


模型训练中：: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [01:28<00:00,  2.90it/s]


train acc: 71.83%, loss: 0.5081


In [57]:
net=load_model('../models/VGG16(224*224)_shuffle_smiling_checkpoint.pth')

In [58]:
CelebA_test(net,target_index=target_index)

全部测试集：
test  acc: 33.74%, loss: 2.0693
性别1测试集：
test  acc: 33.47%, loss: 2.1568
性别2测试集：
test  acc: 34.91%, loss: 1.9318
标签1测试集：
test  acc: 38.15%, loss: 0.7709
标签2测试集：
test  acc: 29.84%, loss: 3.3945


In [63]:
#恢复训练,celebA数据集target为biglips
for epoch in range(10):
        print('epoch: %d' % epoch)
        train(celebA_train_balance_dataloader,net,target_index=target_index,training_type="VGG16(224*224)_balance_smiling")

epoch: 0


模型训练中：: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [01:29<00:00,  2.86it/s]


train acc: 88.19%, loss: 0.2132
epoch: 1


模型训练中：: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [01:28<00:00,  2.87it/s]


train acc: 88.63%, loss: 0.2047
epoch: 2


...

train acc: 89.12%, loss: 0.1880
epoch: 8


模型训练中：: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [01:29<00:00,  2.86it/s]


train acc: 89.14%, loss: 0.1881
epoch: 9


模型训练中：: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [01:28<00:00,  2.87it/s]


train acc: 89.17%, loss: 0.1874


In [61]:
net=load_model('../models/VGG16(224*224)_balance_smiling_checkpoint.pth')

In [64]:
CelebA_test(net,target_index=target_index)

全部测试集：
test  acc: 72.26%, loss: 0.9061
性别1测试集：
test  acc: 70.07%, loss: 0.8734
性别2测试集：
test  acc: 75.74%, loss: 0.9761
标签1测试集：
test  acc: 82.18%, loss: 0.9227
标签2测试集：
test  acc: 61.94%, loss: 0.8948
